In [ ]:
# Requirements
pip install kserve matplotlib
pip install torchvision --index-url https://download.pytorch.org/whl/cu118

In [ ]:
from kserve import RESTConfig, InferenceRESTClient, InferRequest, InferInput
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from random import randrange
import numpy as np
import os

In [ ]:
# Inference
model_name = "dog_im_gen_vllm"
prompt = "Create an image of a realistic dog"

request = InferRequest(
    model_name=model_name,
    infer_inputs=[
        InferInput(name="prompt", shape=[1], datatype="BYTES", data=np.array([prompt.encode('utf-8')]))
    ]
)

cfg = RESTConfig(protocol="v2", verify=False)
client = InferenceRESTClient(cfg)

namespace = "modelserving-demo"
cluster_name = "modelserving-demo"
base_url = f"https://{model_name}-{namespace}.apps.rosa.{cluster_name}.rfa5.p3.openshiftapps.com"

In [ ]:
infer_response = await client.infer(base_url, request, request.model_name, is_graph_endpoint=True)
det_class = dog_breeds_classes[np.argmax(infer_response.outputs[0].data)]
# print("Raw response: ", infer_response)


In [ ]:
# Show generated image
fig = plt.figure(figsize=(1,1))

img = infer_response.outputs[0].data
fig.add_subplot(1, 1, 1)

plt.title("Generated image")
plt.axis("off")
plt.imshow(img.squeeze().permute(1, 2, 0))
plt.show()